## Import packages and set paths

In [39]:
import pandas as pd
import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import matplotlib.pylab as pl
import numpy.ma as ma
import datetime
import math
import scipy.optimize
import random

In [10]:
basepath = '/media/jukes/jukes1/'
sg_path = '/home/jukes/Documents/Sample_glaciers/'

## Read in analysis dates for manual and automated delinations, convert to datetime objs

In [12]:
#read in manual image dates
manual_df= pd.read_csv(basepath+'Manual/Manual_imgdates.csv', sep=',', dtype=str, header=0, usecols=[1,2,3,4])
print(manual_df.shape)
manual_df.head()

(585, 4)


,BoxID,datetimes,Line_x,Line_y
0,001,1987-10-14,"[153, 153, 151, 152, 150, 151, 149, 148, 146, ...","[132, 133, 134, 134, 135, 135, 136, 138, 139, ..."
1,001,1995-04-13,"[152, 153, 153, 153, 153, 153, 154, 154, 155, ...","[135, 136, 137, 138, 139, 140, 141, 142, 142, ..."
2,001,1995-05-31,"[153, 153, 153, 153, 153, 154, 154, 157, 158, ...","[137, 138, 139, 140, 142, 143, 145, 145, 145, ..."
3,001,2013-05-25,"[131, 131, 132, 133, 134, 134, 135, 136, 136, ...","[141, 143, 144, 144, 145, 146, 146, 146, 147, ..."
4,001,2013-05-29,"[130, 131, 130, 131, 131, 132, 134, 134, 135, ...","[142, 142, 144, 144, 145, 146, 147, 148, 148, ..."


In [13]:
#grab the datetime strings
datetimes = manual_df.loc[:,'datetimes']
#create list to store the datetime objects
datetime_objs = []

for date in datetimes:
    date = str(date)
    datetime_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
    datetime_obj = np.datetime64(datetime_obj)
    datetime_objs.append(datetime_obj)

#add the datetime objects back into the datetime column
manual_df['datetimes'] = datetime_objs

In [14]:
#Read in datetags csv as datetime_df
automated_df = pd.read_csv(sg_path+'imgdates.csv', sep=',', dtype=str, header=0, names=['Scene', 'datetimes'])
print(datetime_df.shape)
automated_df.head()

(864, 2)


,Scene,datetimes
510,LC80090132013101LGN01,2013-04-11
1612,LC80090142013101LGN01,2013-04-11
577,LC82330172013102LGN01,2013-04-12
940,LC82330152013102LGN01,2013-04-12
445,LC80080142013110LGN01,2013-04-20


In [15]:
#grab the datetime strings
datetimes = automated_df.loc[:,'datetimes']
#create list to store the datetime objects
datetime_objs = []

for date in datetimes:
    date = str(date)
    datetime_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
    datetime_obj = np.datetime64(datetime_obj)
    datetime_objs.append(datetime_obj)

#add the datetime objects back into the datetime column
automated_df['datetimes'] = datetime_objs

## Find overlaps and select half for testing

In [71]:
overlap_df = manual_df.merge(automated_df, how='inner', on=['datetimes'])
overlap_df = overlap_df.sort_values(by='datetimes', ascending=True)
overlap_df = overlap_df.drop(['Line_x', 'Line_y', 'Scene'], axis=1)
overlap_df = overlap_df.drop_duplicates()
overlap_df.shape

(125, 2)

In [102]:
dates = []

for idx, row in overlap_df.iterrows():
    entry = row['BoxID'] + ','+str(row['datetimes'])[:-9]
#     print(entry)
    dates.append(entry)
# print(dates)

In [92]:
N = len(dates)
print(N)

#pick a random sample of dates for training
train_dates = random.sample(dates, int(N/2)+1)
print(len(train_dates))
# print(train_dates)

#grab remaining for testing
test_dates = []
for date in dates:
    if date not in train_dates:
        test_dates.append(date)
print(len(test_dates))
# print(test_dates)

#Check that they don't overlap, should return empty
print(set(train_dates).intersection(test_dates))

125
63
62
set()


In [100]:
boxes = []
imgdates = []

for td in train_dates:
    BoxID, imgdate = td.split(',')
    
    boxes.append(BoxID)
    imgdates.append(imgdate)

train_df = pd.DataFrame(list(zip(boxes, imgdates)))
#export to csv and text
train_df.to_csv(basepath+'/Manual/train.csv', sep=',', index=False, header=False)
train_df.to_csv(basepath+'/Manual/train.txt', sep=' ', index=False, header=False)

## Define objective function

Example:

In [104]:
def f(x):
    return x**2
minimum = scipy.optimize.fmin(f, 1)
minimum[0]

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 34


-8.881784197001252e-16

#### Our objective function

In [171]:
def center_dist(thresh):
    #Calculate automated tpos
    auto = thresh*1000
    
    #pull in manual tpos
    man = random.choice(np.linspace(600,1000,100))
    
    #return objective function = distance between the two
    return abs(auto-man)

In [192]:
scipy.optimize.fmin(center_dist, 0.8, full_output=True)

(array([0.81]), 0.10101010101016072, 74, 200, 1)

In [196]:
def minimize(guess):
    minimum = scipy.optimize.fmin(center_dist, guess, full_output=True)
    xopt = minimum[0][0]
    funcval = minimum[1]
    return xopt, funcval

In [200]:
thresholds = np.linspace(0.6, 1.0, 100)

t_list = []
m_list = []

for threshold in thresholds:
    t, m = minimize(threshold)
    t_list.append(t)
    m_list.append(m)

results_df = pd.DataFrame(list(zip(t_list, m_list)), columns=['min_t', 'min_f'])

Optimization terminated successfully.
         Current function value: 0.606061
         Iterations: 32
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: 3.509776
         Iterations: 23
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: 0.173552
         Iterations: 21
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: 3.056197
         Iterations: 21
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: 0.833333
         Iterations: 58
         Function evaluations: 155
Optimization terminated successfully.
         Current function value: 0.046017
         Iterations: 71
         Function evaluations: 190
Optimization terminated successfully.
         Current function value: 0.707071
         Iterations: 39
         Function evaluations: 105
Optimization terminated success

Optimization terminated successfully.
         Current function value: 4.712124
         Iterations: 34
         Function evaluations: 90
Optimization terminated successfully.
         Current function value: 1.728946
         Iterations: 37
         Function evaluations: 100
Optimization terminated successfully.
         Current function value: 1.515241
         Iterations: 51
         Function evaluations: 138
Optimization terminated successfully.
         Current function value: 1.090732
         Iterations: 50
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: 1.389250
         Iterations: 36
         Function evaluations: 95
Optimization terminated successfully.
         Current function value: 0.837180
         Iterations: 61
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: 0.449811
         Iterations: 68
         Function evaluations: 185
Optimization terminated succe

In [203]:
# results_df

## MUST BE ABLE TO CALCULATE THE AUTOMATED DELINEATIONS BASED ON ANY THRESHOLD VALUE SET